# COVID-19 data in Poland

plotting data for Poland based on https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

## Preparation of data
to be run once a day

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
#import timeit
#pd.show_versions(as_json=False)

def prepare_data(name, url, i=73):
    df = pd.read_csv(url, parse_dates=True)
    df.rename(columns={'Country/Region': 'Region'}, inplace=True)    
    p = df[df.Region == "Poland"].drop(['Region', 'Province/State', 'Lat', 'Long'], axis=1)
    col = p.rename(index=lambda x: name).transpose()
    col.rename(index=lambda i: datetime.datetime.strptime(i, '%m/%d/%y'), inplace = True)
    return col[col.index >= datetime.datetime(2020, 3, 3)]

def calculate_mean(df, mean_window_size):
    return df.rolling(window=mean_window_size, center=True).mean().rename(columns=lambda x:"mean(%i) %s" % (mean_window_size,x))

#https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
c = prepare_data("confirmed", "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
d = prepare_data("deaths", "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

nc = c.rolling(window=2).apply(lambda x: x[1] - x[0]).rename(columns=lambda x: "new "+x)
nd = d.rolling(window=2).apply(lambda x: x[1] - x[0]).rename(columns=lambda x: "new "+x)

## Playing with data

In [ ]:
def calulate_percentage(df, mean_window_size):
    return df.rolling(window=mean_window_size, center=True).apply(lambda x: x[-1]/x[:-1].mean()).rename(columns=lambda x: "percent (%s)" % (mean_window_size))



ncm2 = calculate_mean(nc, 2)
ncm3 = calculate_mean(nc, 3)
ncm4 = calculate_mean(nc, 4)
ncm7 = calculate_mean(nc, 7)
ncp2 = calulate_percentage(nc, 2)
ncp3 = calulate_percentage(nc, 3)
ncp4 = calulate_percentage(nc, 4)
ncp7 = calulate_percentage(nc, 7)

## Presentation of data

In [ ]:
c.merge(d.merge(nc.merge(ncp3.merge(nd.merge(ncm2, left_index = True, right_index = True), left_index = True, right_index = True), left_index = True, right_index = True), left_index = True, right_index = True), left_index = True, right_index = True).tail(5)

In [ ]:
fig, axes = plt.subplots(nrows=11, ncols=1, sharex=True)
fig.set_size_inches(10,40)
d.plot.bar(ax=axes[0], width=0.8, color='black')
c.plot.bar(ax=axes[1], width=0.8)
nc.plot.bar(ax=axes[2], width=0.8)
ncm2.plot.bar(ax=axes[3], width=0.8, color='g')
ncm3.plot.bar(ax=axes[4], width=0.8, color='g')
ncm4.plot.bar(ax=axes[5], width=0.8, color='g')
ncm7.plot.bar(ax=axes[6], width=0.8, color='g')
ncp2.plot.bar(ax=axes[7], width=0.8, color='g')
ncp3.plot.bar(ax=axes[8], width=0.8, color='g')
ncp4.plot.bar(ax=axes[9], width=0.8, color='g')
ncp7.plot.bar(ax=axes[10], width=0.8, color='g')
#nc.merge(ncm7, left_index = True, right_index = True).plot.bar(ax=axes[5], width=0.8)